# Experiment with automatization

In [21]:
import os
import json
import requests
from dotenv import load_dotenv

In [22]:
is_localhost = True

if is_localhost:
    load_dotenv('../.env')
else:
    load_dotenv()

In [23]:
GRAFANA_URL = "http://localhost:3000"

GRAFANA_USER = os.getenv("GRAFANA_ADMIN_USER")
GRAFANA_PASSWORD = os.getenv("GRAFANA_ADMIN_PASSWORD")

PG_HOST = os.getenv("POSTGRES_HOST")
PG_DB = os.getenv("POSTGRES_DB")
PG_USER = os.getenv("POSTGRES_USER")
PG_PASSWORD = os.getenv("POSTGRES_PASSWORD")
PG_PORT = os.getenv("POSTGRES_PORT")


In [24]:
PG_DB

'glimmerfox_db'

In [26]:
def create_api_key():
    auth = (GRAFANA_USER, GRAFANA_PASSWORD)
    headers = {"Content-Type": "application/json"}
    payload = {
        "name": "ProgrammaticKey",
        "role": "Admin",
    }
    response = requests.post(
        f"{GRAFANA_URL}/api/auth/keys", auth=auth, headers=headers, json=payload
    )

    if response.status_code == 200:
        print("API key created successfully")
        return response.json()["key"]

    elif response.status_code == 409:  # Conflict, key already exists
        print("API key already exists, updating...")
        # Find the existing key
        keys_response = requests.get(f"{GRAFANA_URL}/api/auth/keys", auth=auth)
        if keys_response.status_code == 200:
            for key in keys_response.json():
                if key["name"] == "ProgrammaticKey":
                    # Delete the existing key
                    delete_response = requests.delete(
                        f"{GRAFANA_URL}/api/auth/keys/{key['id']}", auth=auth
                    )
                    if delete_response.status_code == 200:
                        print("Existing key deleted")
                        # Create a new key
                        return create_api_key()
        print("Failed to update API key")
        return None
    else:
        print(f"Failed to create API key: {response.text}")
        return None


In [27]:
def create_or_update_datasource(api_key):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    datasource_payload = {
        "name": "PostgreSQL",
        "type": "postgres",
        "url": f"{PG_HOST}:{PG_PORT}",
        "access": "proxy",
        "user": PG_USER,
        "database": PG_DB,
        "basicAuth": False,
        "isDefault": True,
        "jsonData": {"sslmode": "disable", "postgresVersion": 1300},
        "secureJsonData": {"password": PG_PASSWORD},
    }

    print("Datasource payload:")
    print(json.dumps(datasource_payload, indent=2))

    # First, try to get the existing datasource
    response = requests.get(
        f"{GRAFANA_URL}/api/datasources/name/{datasource_payload['name']}",
        headers=headers,
    )

    if response.status_code == 200:
        # Datasource exists, let's update it
        existing_datasource = response.json()
        datasource_id = existing_datasource["id"]
        print(f"Updating existing datasource with id: {datasource_id}")
        response = requests.put(
            f"{GRAFANA_URL}/api/datasources/{datasource_id}",
            headers=headers,
            json=datasource_payload,
        )
    else:
        # Datasource doesn't exist, create a new one
        print("Creating new datasource")
        response = requests.post(
            f"{GRAFANA_URL}/api/datasources", headers=headers, json=datasource_payload
        )

    print(f"Response status code: {response.status_code}")
    print(f"Response headers: {response.headers}")
    print(f"Response content: {response.text}")

    if response.status_code in [200, 201]:
        print("Datasource created or updated successfully")
        return response.json().get("datasource", {}).get("uid") or response.json().get(
            "uid"
        )
    else:
        print(f"Failed to create or update datasource: {response.text}")
        return None

In [28]:
def create_dashboard(api_key, datasource_uid):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    if is_localhost:
        dashboard_file = "../grafana/glimmerfox-dashboard.json"
    else:
        dashboard_file = "glimmerfox-dashboard.json"

    try:
        with open(dashboard_file, "r") as f:
            dashboard_json = json.load(f)
    except FileNotFoundError:
        print(f"Error: {dashboard_file} not found.")
        return
    except json.JSONDecodeError as e:
        print(f"Error decoding {dashboard_file}: {str(e)}")
        return

    print("Dashboard JSON loaded successfully.")

    # Update datasource UID in the dashboard JSON
    panels_updated = 0
    for panel in dashboard_json.get("panels", []):
        if isinstance(panel.get("datasource"), dict):
            panel["datasource"]["uid"] = datasource_uid
            panels_updated += 1
        elif isinstance(panel.get("targets"), list):
            for target in panel["targets"]:
                if isinstance(target.get("datasource"), dict):
                    target["datasource"]["uid"] = datasource_uid
                    panels_updated += 1

    print(f"Updated datasource UID for {panels_updated} panels/targets.")

    # Remove keys that shouldn't be included when creating a new dashboard
    dashboard_json.pop("id", None)
    dashboard_json.pop("uid", None)
    dashboard_json.pop("version", None)

    # Prepare the payload
    dashboard_payload = {
        "dashboard": dashboard_json,
        "overwrite": True,
        "message": "Updated by Python script",
    }

    print("Sending dashboard creation request...")

    response = requests.post(
        f"{GRAFANA_URL}/api/dashboards/db", headers=headers, json=dashboard_payload
    )

    print(f"Response status code: {response.status_code}")
    print(f"Response content: {response.text}")

    if response.status_code == 200:
        print("Dashboard created successfully")
        return response.json().get("uid")
    else:
        print(f"Failed to create dashboard: {response.text}")
        return None

In [ ]:
api_key = create_api_key()
if not api_key:
    print("API key creation failed")
# api_key

In [30]:
datasource_uid = create_or_update_datasource(api_key)
if not datasource_uid:
    print("Datasource creation failed")
datasource_uid


Datasource payload:
{
  "name": "PostgreSQL",
  "type": "postgres",
  "url": "postgres:5432",
  "access": "proxy",
  "user": "your_username",
  "database": "glimmerfox_db",
  "basicAuth": false,
  "isDefault": true,
  "jsonData": {
    "sslmode": "disable",
    "postgresVersion": 1300
  },
  "secureJsonData": {
    "password": "your_password"
  }
}
Updating existing datasource with id: 2
Response status code: 200
Response headers: {'Cache-Control': 'no-store', 'Content-Type': 'application/json', 'X-Content-Type-Options': 'nosniff', 'X-Xss-Protection': '1; mode=block', 'Date': 'Sat, 14 Sep 2024 13:34:57 GMT', 'Content-Length': '578'}
Response content: {"datasource":{"id":2,"uid":"bdxvgm04xh79ca","orgId":1,"name":"PostgreSQL","type":"grafana-postgresql-datasource","typeLogoUrl":"public/app/plugins/datasource/grafana-postgresql-datasource/img/postgresql_logo.svg","access":"proxy","url":"postgres:5432","user":"your_username","database":"glimmerfox_db","basicAuth":false,"basicAuthUser":"","

'bdxvgm04xh79ca'

In [31]:
create_dashboard(api_key, datasource_uid)

Error: ../grafana/glimmerfox-dashboard.json not found.


In [32]:
def check_grafana_status(url="http://localhost:3000"):
    try:
        # Send a GET request to the Grafana server
        response = requests.get(url)

        # Check if the response status code is 200 (OK)
        if response.status_code == 200:
            print("Grafana service is running.")
        else:
            print(f"Grafana service returned status code {response.status_code}. It may not be running correctly.")
    except requests.exceptions.RequestException as e:
        print(f"Error connecting to Grafana service: {e}")

In [33]:
check_grafana_status()

Grafana service is running.
